# Interactive Viz

In [24]:
import googlemaps
import folium
import pandas as pd
import requests
import os.path
import json
import html5lib
from bs4 import BeautifulSoup
import re

In [2]:
GOOGLE_KEY_PATH = './google_key'
GOOGLE_KEY = None
with open(GOOGLE_KEY_PATH, 'r') as content_file:
    GOOGLE_KEY = content_file.read()

First thing first, lets import the data

In [3]:
data = pd.read_csv('Data/P3_GrantExport.csv',sep=';')
data.set_index(data.columns[0], inplace=True)
print(data.shape)
data.head(100)

(63969, 14)


,Project Title,Project Title English,Responsible Applicant,Funding Instrument,Funding Instrument Hierarchy,Institution,University,Discipline Number,Discipline Name,Discipline Name Hierarchy,Start Date,End Date,Approved Amount,Keywords
"﻿""Project Number""",,,,,,,,,,,,,,
1,Schlussband (Bd. VI) der Jacob Burckhardt-Biog...,NaN,Kaegi Werner,Project funding (Div. I-III),Project funding,NaN,Nicht zuteilbar - NA,10302,Swiss history,Human and Social Sciences;Theology & religious...,01.10.1975,30.09.1976,11619.00,NaN
4,Batterie de tests à l'usage des enseignants po...,NaN,Massarenti Léonard,Project funding (Div. I-III),Project funding,Faculté de Psychologie et des Sciences de l'Ed...,Université de Genève - GE,10104,Educational science and Pedagogy,"Human and Social Sciences;Psychology, educatio...",01.10.1975,30.09.1976,41022.00,NaN
5,"Kritische Erstausgabe der ""Evidentiae contra D...",NaN,Kommission für das Corpus philosophorum medii ...,Project funding (Div. I-III),Project funding,Kommission für das Corpus philosophorum medii ...,"NPO (Biblioth., Museen, Verwalt.) - NPO",10101,Philosophy,Human and Social Sciences;Linguistics and lite...,01.03.1976,28.02.1985,79732.00,NaN
6,Katalog der datierten Handschriften in der Sch...,NaN,Burckhardt Max,Project funding (Div. I-III),Project funding,Abt. Handschriften und Alte Drucke Bibliothek ...,Universität Basel - BS,10302,Swiss history,Human and Social Sciences;Theology & religious...,01.10.1975,30.09.1976,52627.00,NaN
7,Wissenschaftliche Mitarbeit am Thesaurus Lingu...,NaN,Schweiz. Thesauruskommission,Project funding (Div. I-III),Project funding,Schweiz. Thesauruskommission,"NPO (Biblioth., Museen, Verwalt.) - NPO",10303,Ancient history and Classical studies,Human and Social Sciences;Theology & religious...,01.01.1976,30.04.1978,120042.00,NaN
8,Die schweizerische Wirtschaftspolitik seit dem...,NaN,Kleinewefers Henner,Project funding (Div. I-III),Project funding,"Séminaire de politique économique, d'économie ...",Université de Fribourg - FR,10203,Economics,"Human and Social Sciences;Economics, law",01.01.1976,31.12.1978,53009.00,NaN
9,Theologische Forschungen zur Oekumene (Studien...,NaN,Stirnimann Heinrich,Project funding (Div. I-III),Project funding,Institut für ökumenische Studien Université de...,Université de Fribourg - FR,10102,"Religious sciences, Theology",Human and Social Sciences;Theology & religious...,01.01.1976,31.12.1976,25403.00,NaN
10,Konfuzianische Kulturwerte in der sozialen Ent...,NaN,Deuchler Martina,Project funding (Div. I-III),Project funding,Ostasiatisches Seminar Universität Zürich,Universität Zürich - ZH,10301,History in general,Human and Social Sciences;Theology & religious...,01.10.1975,31.03.1977,47100.00,NaN
11,Edizione degli scritti di Aurelio de' Giorgi B...,NaN,Stäuble Antonio,Project funding (Div. I-III),Project funding,NaN,Université de Lausanne - LA,10502,Romance languages and literature,Human and Social Sciences;Linguistics and lite...,01.10.1975,31.03.1977,25814.00,NaN


Then we keep only the columns we need (hence the university and the approved amount).
Plus, we remove the rows where no university is specified.

In [4]:
data = data[['University', 'Approved Amount']]
data.dropna(subset=['University'], inplace=True)
data = data.ix[data['University'] != 'Nicht zuteilbar - NA'] #NA university
data = data.ix[data['University'] != 'NPO (Biblioth., Museen, Verwalt.) - NPO']
print(data.shape)
data.head()

(46920, 2)


,University,Approved Amount
"﻿""Project Number""",,
4,Université de Genève - GE,41022.00
6,Universität Basel - BS,52627.00
8,Université de Fribourg - FR,53009.00
9,Université de Fribourg - FR,25403.00
10,Universität Zürich - ZH,47100.00


The next step is to map each university to its canton. We first tried to use GeoNames, and then Google, but they all failed to provide us the panacea. That's why we are directly querying P3 (thanks beautiful soup).

In [61]:
GEONAMES_NPA_URL = 'http://api.geonames.org/postalCodeSearchJSON?country=CH&maxRows=1&username=luc4s'

P3_BASE_URL = 'http://p3.snf.ch'
P3_PROJECT = P3_BASE_URL + '/project-'
P3_INSTITUTE = P3_BASE_URL + '/institute-274'
    
UNIVERSITIES_CANTONS = 'Data/universities_cantons.xls'

def get_canton(idx):
    r = requests.get(P3_PROJECT + str(idx))
    soup = BeautifulSoup(r.text, 'html.parser')
    institute = soup.find(id="ctl00_MainContent_urlInstitite") #found it manually
    
    if institute == None:
        return None
    
    #headers = {'user-agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:49.0) Gecko/20100101 Firefox/49.0'}
    
    payload = {'number': idx}

    r = requests.post(P3_INSTITUTE, data=payload)
    soup = BeautifulSoup(r.text, "html5lib")
    address_soup = soup.find_all('div', attrs={'class' : 'institute'})[0].table.tbody
    postal_code = re.compile("CH\-[0-9]{4}").findall(address_soup.prettify())
    if(len(postal_code) < 1):
        return None #shouldn't happen, but you never know
    
    postal_code = int(postal_code[0][3:])
    print(str(idx) + ", npa:" + str(postal_code))
    
    #Now that we have the postal code, we can use GeoNames to get the cantons, and we're done!
    payload = {'postalcode': postal_code}
    r = requests.get(GEONAMES_NPA_URL, params=payload)
    res = r.json()
    if(len(res['postalCodes']) < 1):
        return None #just in case
    
    return res['postalCodes'][0]['adminCode1']
    
def get_uni_cantons(update):
    if(not os.path.isfile(UNIVERSITIES_CANTONS) or update):
        #we do this so we don't have to redo the mapping every time
        universities = data['University'].drop_duplicates().to_frame()
        
        universities['Canton'] = universities.index.map(lambda x: get_canton(x))
        
        universities.to_excel(UNIVERSITIES_CANTONS)
        return universities
    else:
        return pd.read_excel(UNIVERSITIES_CANTONS)

uni_cantons = get_uni_cantons(True)


4, npa:1211
6, npa:1211
8, npa:1211
10, npa:1211
11, npa:1211
30, npa:1211


KeyboardInterrupt: 

In [6]:
gmaps = googlemaps.Client(key=GOOGLE_KEY)

def get_canton(university):
    value = university.split(" - ")
    geocode_result = gmaps.geocode(value)
    if len(geocode_result) > 0:
        for n in geocode_result[0]['address_components'] :
            if(n['types'][0] == 'administrative_area_level_1' ):
                    print(n['short_name'])

unit_cantons = get_uni_cantons(True)

AttributeError: 'DataFrame' object has no attribute 'get_loc'

In [ ]:
ch_geo = r'ch-cantons.topojson.json'

p3data = pd.DataFrame({'Canton':['JU', 'ZH', 'BE', 'LU', 'SG', 'NE', 'VD', 'VS', 'GE', 'TI', 
                                 'SO', 'UR', 'OW', 'NW', 'SZ', 'GL', 'ZG', 'BL', 'BS', 'FR',
                                'SH', 'AR', 'AI', 'GR', 'AG', 'TG']})
p3data['Grants'] = 0
p3data.loc[0, 'Grants'] = 1000

map = folium.Map(location=[46.82244,8.22410], zoom_start=8)
map.choropleth(data=p3data, 
                columns=['Canton', 'Grants'], 
                geo_path=ch_geo, 
                topojson='objects.cantons', 
                fill_color='YlOrRd', fill_opacity=0.7, line_opacity=0.2,
                key_on='feature.id')

map.save('ch_map.html')
map